In [492]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import matplotlib.pyplot as plt

In [494]:
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True


In [495]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
site_name = spacex_df["Launch Site"].value_counts().index.tolist()


In [496]:
payload_df = spacex_df[ [ "Launch Site", "class", "Payload Mass (kg)" ] ].loc[ ( spacex_df [ "Payload Mass (kg)" ] >= float(500) ) & 
                  ( spacex_df [ "Payload Mass (kg)" ] < float(600) ) ]
print(payload_df)
payload_df.loc[ payload_df["Launch Site"] == "CCAFS LC-40"]


    Launch Site  class  Payload Mass (kg)
2   CCAFS LC-40      0              525.0
3   CCAFS LC-40      0              500.0
13  CCAFS LC-40      0              570.0
26  VAFB SLC-4E      0              500.0
27  VAFB SLC-4E      0              553.0


,Launch Site,class,Payload Mass (kg)
2,CCAFS LC-40,0,525.0
3,CCAFS LC-40,0,500.0
13,CCAFS LC-40,0,570.0


In [497]:
def get_graph(name, payload1, payload2):
    payload_df = spacex_df[ [ "Launch Site", "class", "Payload Mass (kg)" ] ].loc[ ( spacex_df [ "Payload Mass (kg)" ] >= float(payload1) ) & 
                  ( spacex_df [ "Payload Mass (kg)" ] < float(payload2) ) ]

    newpdf = payload_df.loc[ payload_df["Launch Site"] == name ]
    figr = px.scatter(newpdf, x = "Payload Mass (kg)", y="class", color = "class",
                     title="Scatter plot for apyload  mass vs class" )
    figr.update_traces(marker_coloraxis=None)                         
    return figr

In [498]:
class_count = spacex_df.groupby(['Launch Site'])['class'].value_counts() 
class_count2 = list(class_count)
class_name = class_count.index.tolist()
print(class_count)
print(class_name)
print(class_count2)
new_df = pd.DataFrame()
new_df["name"] = class_name
new_df["val"] = class_count2
new_df.head()

Launch Site   class
CCAFS LC-40   0        19
              1         7
CCAFS SLC-40  0         4
              1         3
KSC LC-39A    1        10
              0         3
VAFB SLC-4E   0         6
              1         4
Name: class, dtype: int64
[('CCAFS LC-40', 0), ('CCAFS LC-40', 1), ('CCAFS SLC-40', 0), ('CCAFS SLC-40', 1), ('KSC LC-39A', 1), ('KSC LC-39A', 0), ('VAFB SLC-4E', 0), ('VAFB SLC-4E', 1)]
[19, 7, 4, 3, 10, 3, 6, 4]


,name,val
0,"(CCAFS LC-40, 0)",19
1,"(CCAFS LC-40, 1)",7
2,"(CCAFS SLC-40, 0)",4
3,"(CCAFS SLC-40, 1)",3
4,"(KSC LC-39A, 1)",10


In [500]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', 
                                            # Update dropdown values using list comphrehension
                                            options=[{'label':i, 'value': i} for i in site_name],
                                            placeholder = "Select a Launch Site",
                                            style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),

                          
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')), 
                                             
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', min = 0, max = 1000, step = 100, value=[min_payload, max_payload] ),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id = 'success-pie-chart',component_property = 'figure'),
    [Input(component_id = 'site-dropdown', component_property = 'value')], 
    prevent_initial_call = True
)


def generate_chart(name):
    
    if name == "CCAFS LC-40":
        df = new_df.iloc[0:2]
        fig = px.pie(df, values = 'val', names = 'name', title = "Pie chart plot of Success vs failure")
        return fig
    
    elif name == "KSC LC-39A":
        df = new_df.iloc[4:6]
        fig = px.pie(df, values = 'val', names = 'name', title = "Pie chart plot of Success vs failure")
        return fig
    
    elif name == "VAFB SLC-4E":
        df = new_df.iloc[6:8]
        fig = px.pie(df, values = 'val', names = 'name', title = "Pie chart plot of Success vs failure")
        return fig
    
    else:
        df = new_df.iloc[2:4]
        fig = px.pie(df, values = 'val', names = 'name', title = "Pie chart plot of Success vs failure")
        return fig
      
              
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id = 'success-payload-scatter-chart',component_property = 'figure'),
    [Input(component_id = 'site-dropdown', component_property = 'value'),
     Input(component_id = 'payload-slider', component_property = 'value')], 
    prevent_initial_call = True
)


def get_graph(name, payload):
    payload_df = spacex_df[ [ "Launch Site", "class", "Payload Mass (kg)" ] ].loc[ ( spacex_df [ "Payload Mass (kg)" ] >= float(payload[0]) ) & 
                  ( spacex_df [ "Payload Mass (kg)" ] < float(payload[1]) ) ]

    newpdf = payload_df.loc[ payload_df["Launch Site"] == name ]
    figr = px.scatter(newpdf, x = "Payload Mass (kg)", y="class", color = "class",
                     title="Scatter plot for apyload  mass vs class" )
    figr.update_traces(marker_coloraxis=None)                         
    return figr
    
    

# Run the app
if __name__ == '__main__':
    app.run_server( mode="inline", host="localhost", port=8060, debug=False, dev_tools_ui=False, dev_tools_props_check=False)

127.0.0.1 - - [01/Sep/2021 16:43:51] "GET /_shutdown_46174835-42e5-469c-af94-b5dd1b84215e HTTP/1.1" 200 -
 * Running on http://localhost:8060/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Sep/2021 16:43:51] "GET /_alive_46174835-42e5-469c-af94-b5dd1b84215e HTTP/1.1" 200 -


127.0.0.1 - - [01/Sep/2021 16:43:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 16:43:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 16:43:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 16:43:52] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 16:43:52] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 16:43:52] "GET /_dash-component-suites/dash_core_components/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 16:43:52] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
